# Ollama PDF RAG

## Import Libraries

In [ ]:
from click import prompt
from langchain_community.document_loaders import UnstructuredPDFLoader
from langchain_ollama import OllamaEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import PGVector
from langchain_community.vectorstores import Chroma
from langchain.prompts import ChatPromptTemplate, PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_ollama.chat_models import ChatOllama
from langchain_core.runnables import RunnablePassthrough
from langchain.retrievers.multi_query import MultiQueryRetriever

#suppress warnings
import warnings
warnings.filterwarnings('ignore')

from IPython.display import display, Markdown

import os 
os.environ["PROTOCOL_BUFFERS_PYTHON_IMPLEMETATION"] = "python"


## Load PDFs

In [5]:
load_path = "/Users/donaldparker/Projects/lab/rag/ollama-pgvector/pdfs/tacoma-2019.pdf"
if load_path:
    loader = UnstructuredPDFLoader(file_path=load_path)
    pdf_data = loader.load()
    print(f"PDF Loaded successfully: {load_path}")
else:
    print(f"Unable to load PDF")

PDF Loaded successfully: /Users/donaldparker/Projects/lab/rag/ollama-pgvector/pdfs/tacoma-2019.pdf


## Split text into chunks

In [6]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
chunks = text_splitter.split_documents(pdf_data)
print(f"Text split into {len(chunks)} chunks")

Text split into 986 chunks


## Vector Database

In [7]:
chroma_db = Chroma.from_documents(
    documents=chunks,
    embedding=OllamaEmbeddings(model="nomic-embed-text"),
    collection_name="local-rag-tacoma"
)
print(f"Vector database created successfully")

Vector database created successfully


## Set LLM and Retrieval

In [30]:
local_model = "llama3.2:latest"
#local_model = "granite3-dense:8b"
llm = ChatOllama(model=local_model)

query_prompt = PromptTemplate(
    input_variables=["question"],
    template="""You are an AI language model assistant.  Your task is to generate 2 
    different versions of the give user question to retrieve relevant documents from
    a vector database. By generating multiple perspectives on user question, your
    goal is to help users overcome some of the limitations of distance-based
    similarity search. Provide these alternative questions separated by newlines.
    Original question: {question}"""
)
retriever = MultiQueryRetriever.from_llm(
    chroma_db.as_retriever(),
    llm,
    prompt=query_prompt
)

## Create Chain

In [31]:
template = """Answer the question on on the following context: 
{context}
Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)

In [32]:
chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [33]:
def chat_with_pdf(question): 
    """
    Chat with the PDF using our chain
    """
    return display(Markdown(chain.invoke(question)))

In [35]:
chat_with_pdf("How to I connect my phone to bluetooth in my tacoma?")

According to the manual, to connect a Bluetooth device to your Tacoma:

1. Go to the "Bluetooth* Audio" screen and select "Select Device".
2. Display the "Bluetooth* Setup" screen and select the device you want to connect (your phone).
3. Select "Device Info" and then "Restore default settings" if needed.
4. If prompted, change the connection method:
	* Select "Connect Audio Player from" and choose either "Vehicle" or "Device".
	* If you want to connect your phone to the vehicle's audio system, select "Vehicle". If you want to connect your audio player to the vehicle's Bluetooth system, select "Device".
5. To display detailed Bluetooth settings, go back to the "Bluetooth* Setup" screen and select "System Settings".

Before making a call, make sure to check:

* The connection status
* Battery charge
* Call area (which should be set to "Hands Free" or "Phone" mode)
* Signal strength

To operate on each screen that is displayed, press the steering wheel button or the "phone" button. Several functions are available for each screen by selecting the 4 tabs.

Note: If you have a Tacoma with a moonroof, the built-in microphone will be used when talking on the phone, and you must register your Bluetooth device to use the hands-free system.

In [29]:
#granite 
chat_with_pdf("How to I connect my phone to bluetooth in my tacoma?")

To connect your phone to Bluetooth in your Tacoma, follow these steps:

1. Display the "Bluetooth* Setup" screen by pressing the "SETUP" button and selecting "Bluetooth" on the "Setup" screen.
2. Select to connect the device you want to use with the multimedia system.
3. Once the phone is connected, you can use the Bluetooth Audio feature. To turn auto connection mode on, set "Bluetooth* Power" to on. When you register a phone, auto connection will be activated. Always set it to this mode and leave the Bluetooth phone in a place where a connection can be established.

## Cleanup

In [36]:
chroma_db.delete_collection()
print("Vector database deleted successfully")

Vector database deleted successfully
